In [4]:
# Use Google Colab
use_colab = True

# Is this notebook running on Colab?
# If so, then google.colab package (github.com/googlecolab/colabtools)
# should be available in this environment

# Previous version used importlib, but we could do the same thing with
# just attempting to import google.colab
try:
    from google.colab import drive
    colab_available = True
except:
    colab_available = False

if use_colab and colab_available:
    drive.mount('/content/drive')

    # cd to the appropriate working directory under my Google Drive
    %cd '/content/drive/My Drive/cs696ds_lexalytics/Ronald Gypsum Prompts'
    
    # Install packages specified in requirements
    !pip install -r requirements.txt

    %cd 'prompts_subtask4'
    
    # List the directory contents
    !ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/cs696ds_lexalytics/Ronald Gypsum Prompts
/content/drive/My Drive/cs696ds_lexalytics/Ronald Gypsum Prompts/prompts_subtask4
bert_no_prompt_cls_lr_subtask4.ipynb
bert_no_prompt_pooled_lr_subtask4.ipynb
bert_prompt_logit_softmax_subtask4.ipynb
bert_prompt_lr_subtask4.ipynb
collect_results_snippet.ipynb
papermill_launcher_no_prompt_cls_lr.ipynb
papermill_launcher_no_prompt_pooled_lr.ipynb
papermill_launcher_prompt_logit_softmax.ipynb
papermill_launcher_prompt_lr.ipynb
results_bert_16_shot_no_prompt_cls_lr_atsc_restaurants
results_bert_16_shot_no_prompt_pooled_lr_atsc_restaurants
results_bert_16_shot_prompt_logit_softmax_atsc_restaurants
results_bert_16_shot_prompt_lr_concatenate_atsc_restaurants
results_bert_256_shot_no_prompt_cls_lr_atsc_restaurants
results_bert_256_shot_no_prompt_pooled_lr_atsc_restaurants
results_bert_256_shot_prompt_lo

In [1]:
import os
import itertools

import papermill
import tqdm

In [10]:
# experiment id prefix
experiment_id_prefix = 'bert_prompt_logit_softmax_subtask4'

# Random seed
random_seeds = [696]

# path to pretrained MLM model folder or the string "bert-base-uncased"
lm_model_paths = {
    'bert_yelp_restaurants': '../trained_models/lm_further_pretraining_bert_yelp_restaurants_bseoh_2021-03-22--15_03_31',
    #'bert-base-uncased': 'bert-base-uncased'
}

# Prompts to be added to the end of each review text
# Note: pseudo-labels for each prompt should be given in the order of (positive), (negative), (neutral)
sentiment_prompts = {
    'i_felt': {"prompt": "I felt the {aspect} was [MASK].", "labels": ["good", "bad", "ok"]},
    'i_like': {"prompt": "I [MASK] the {aspect}.", "labels": ["like", "dislike", "ignore"]},
    'made_me_feel': {"prompt": "The {aspect} made me feel [MASK].", "labels": ["good", "bad", "indifferent"]},
    'the_aspect_is': {"prompt": "The {aspect} is [MASK].", "labels": ["good", "bad", "ok"]}
}

run_single_prompt = True
run_multiple_prompts = True

prompts_merge_behavior = 'sum_logits'
prompts_perturb = False

no_atsc_training = True

# Training settings
training_domain = 'restaurants' # 'laptops', 'restaurants', 'joint'

# Few-shot dataset size
training_dataset_few_shot_size = 1024

if not no_atsc_training:
    if training_dataset_few_shot_size == -1:
        experiment_id_prefix_override = 'bert_prompt_logit_softmax_atsc'
    else:
        experiment_id_prefix_override = 'bert_' + str(training_dataset_few_shot_size) + '_shot_' + 'prompt_logit_softmax_sum_logits_atsc'
else:
    experiment_id_prefix_override = 'bert_zero_shot_prompt_logit_softmax_atsc'

# Test settings
testing_batch_size = 32
testing_domain = 'restaurants'

if testing_domain != training_domain:
    cross_domain = True
else:
    cross_domain = False

# Results directory path
results_path = 'results_' + experiment_id_prefix_override + '_' + testing_domain
os.makedirs(results_path, exist_ok=True)

In [11]:
# Run single prompt experiments first
if run_single_prompt:
    
    print("Running single prompts...")
    print()

    for seed, lm_model_name, prompt_key in tqdm.tqdm(itertools.product(random_seeds, lm_model_paths.keys(), sentiment_prompts.keys())):
        
        # We will use the following string ID to identify this particular (training) experiments
        # in directory paths and other settings
        experiment_id = experiment_id_prefix_override + '_'
        experiment_id = experiment_id + testing_domain + '_'
        
        if cross_domain:
            experiment_id = experiment_id + 'cross_domain_'

        experiment_id = experiment_id + lm_model_name + '_'
        experiment_id = experiment_id + 'single_prompt' + '_'
        experiment_id = experiment_id + prompt_key + '_'
        experiment_id = experiment_id + str(seed)
        
        print("Running experiment", experiment_id)
        print()
        
        parameters_to_inject = {
            'experiment_id': experiment_id,
            'random_seed': seed,
            'lm_model_path': lm_model_paths[lm_model_name],
            #'training_domain': training_domain,
            'sentiment_prompts': [sentiment_prompts[prompt_key]],
            #'training_dataset_few_shot_size': training_dataset_few_shot_size,
            'testing_batch_size': testing_batch_size,
            'testing_domain': testing_domain,
            'prompts_merge_behavior': prompts_merge_behavior,
            'prompts_perturb': prompts_perturb,
            'no_atsc_training': no_atsc_training
        }

        papermill.execute_notebook(
            os.path.join(experiment_id_prefix + '.ipynb'),
            os.path.join(results_path, experiment_id + '.ipynb'),
            parameters=parameters_to_inject,
            log_output=True,
            progress_bar=True,
            autosave_cell_every=300,
            cwd=os.getcwd(),
        )

0it [00:00, ?it/s]

Running single prompts...

Running experiment bert_zero_shot_prompt_logit_softmax_atsc_restaurants_bert_yelp_restaurants_single_prompt_i_felt_696



Executing:   0%|          | 0/32 [00:00<?, ?cell/s]

Intel(R) oneAPI Data Analytics Library solvers for sklearn enabled: https://intelpython.github.io/daal4py/sklearn.html

Reusing dataset sem_eval2014_task4_dataset (../dataset_cache/sem_eval2014_task4_dataset/SemEval2014Task4Dataset - Subtask 4/0.0.1/537edd3b5fdbdb1f3190419cf0a53a4fab3537bc666f17c8c75fa8d0b554e529)

1it [00:33, 33.23s/it]

Running experiment bert_zero_shot_prompt_logit_softmax_atsc_restaurants_bert_yelp_restaurants_single_prompt_i_like_696



...

Executing:   0%|          | 0/32 [00:00<?, ?cell/s]

Intel(R) oneAPI Data Analytics Library solvers for sklearn enabled: https://intelpython.github.io/daal4py/sklearn.html

Reusing dataset sem_eval2014_task4_dataset (../dataset_cache/sem_eval2014_task4_dataset/SemEval2014Task4Dataset - Subtask 4/0.0.1/537edd3b5fdbdb1f3190419cf0a53a4fab3537bc666f17c8c75fa8d0b554e529)

/home/bseoh/miniconda3/envs/zeroshotatsc/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))

3it [01:24, 27.34s/it]

Running experiment bert_zero_shot_prompt_logit_softmax_atsc_restaurants_bert_yelp_restaurants_single_prompt_the_aspect_is_696



Executing:   0%|          | 0/32 [00:00<?, ?cell/s]

Intel(R) oneAPI Data Analytics Library solvers for sklearn enabled: https://intelpython.github.io/daal4py/sklearn.html

Reusing dataset sem_eval2014_task4_dataset (../dataset_cache/sem_eval2014_task4_dataset/SemEval2014Task4Dataset - Subtask 4/0.0.1/537edd3b5fdbdb1f3190419cf0a53a4fab3537bc666f17c8c75fa8d0b554e529)

4it [01:50, 27.74s/it]


In [12]:
# Run multiple prompts
if run_multiple_prompts:
    print("Running multiple prompts...")
    print()

    for seed, lm_model_name in tqdm.tqdm(itertools.product(random_seeds, lm_model_paths.keys())):
        
        # We will use the following string ID to identify this particular (training) experiments
        # in directory paths and other settings
        experiment_id = experiment_id_prefix_override + '_'
        experiment_id = experiment_id + testing_domain + '_'
        
        if cross_domain:
            experiment_id = experiment_id + 'cross_domain_'

        experiment_id = experiment_id + lm_model_name + '_'
        experiment_id = experiment_id + 'multiple_prompts' + '_'
        experiment_id = experiment_id + str(seed)
        
        print("Running experiment ", experiment_id)
        print()
        
        parameters_to_inject = {
            'experiment_id': experiment_id,
            'random_seed': seed,
            'lm_model_path': lm_model_paths[lm_model_name],
            'sentiment_prompts': [sentiment_prompts[prompt_key] for prompt_key in sentiment_prompts.keys()],
            #'training_domain': training_domain,
            #'training_dataset_few_shot_size': training_dataset_few_shot_size,
            'testing_batch_size': testing_batch_size,
            'testing_domain': testing_domain,
            'prompts_merge_behavior': prompts_merge_behavior,
            'prompts_perturb': prompts_perturb,
            'no_atsc_training': no_atsc_training
        }

        papermill.execute_notebook(
            os.path.join(experiment_id_prefix + '.ipynb'),
            os.path.join(results_path, experiment_id + '.ipynb'),
            parameters=parameters_to_inject,
            log_output=True,
            progress_bar=True,
            autosave_cell_every=300,
            cwd=os.getcwd(),
        )

0it [00:00, ?it/s]

Running multiple prompts...

Running experiment  bert_zero_shot_prompt_logit_softmax_atsc_restaurants_bert_yelp_restaurants_multiple_prompts_696



Executing:   0%|          | 0/32 [00:00<?, ?cell/s]

Intel(R) oneAPI Data Analytics Library solvers for sklearn enabled: https://intelpython.github.io/daal4py/sklearn.html

Reusing dataset sem_eval2014_task4_dataset (../dataset_cache/sem_eval2014_task4_dataset/SemEval2014Task4Dataset - Subtask 4/0.0.1/537edd3b5fdbdb1f3190419cf0a53a4fab3537bc666f17c8c75fa8d0b554e529)

/home/bseoh/miniconda3/envs/zeroshotatsc/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))

1it [00:52, 52.25s/it]
